In [65]:
import os
from dotenv import load_dotenv
import json
from openai import OpenAI
import gradio as gr
import anthropic
import base64
from io import BytesIO
from PIL import Image
from pydub import AudioSegment
from pydub.playback import play


In [66]:
load_dotenv(override=True)
openai_key = os.getenv('OPENAI_API_KEY')

In [67]:
openai = OpenAI()

In [68]:
openai_model = 'gpt-4o-mini'

In [69]:
system_message = "You are an helpful AI assistant for an airline called topairline \
                Give short polite message, not more than 1 sentence \
                Always be accurate. If you don't know the answer, say so."


In [70]:
ticket_prices = {
    "london": "$799", 
    "paris": "$899", 
    "tokyo": "$1400", 
    "berlin": "$499", 
    "new york": "$1200", 
    "sydney": "$1500", 
    "rome": "$950", 
    "mumbai": "$850", 
    "barcelona": "$1100", 
    "amsterdam": "$1050", 
    "dubai": "$1300", 
    "bangkok": "$750", 
    "cape town": "$1600"
}

In [71]:
class AirlineAssistant:

    def __init__(self, destination_city):
        self.system_message = system_message
        self.ticket_prices = ticket_prices
        self.destination_city = destination_city

    
    def get_ticket_prices(self):
        print(f"Tool get_ticket_prices for {self.destination_city}")
        city = self.destination_city.lower()
        return ticket_prices.get(city, "Unknown")

    
    def price_function(self):
        price_function= {
            'name' : 'get_ticket_prices',
            'description' : "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
            "parameters" : {
                "type" : "object",
                "properties" : {
                    "destination_city" : {
                        "type" : "string",
                        "description" : "The city that the customer wants to travel to",
                    },
                },
                    "required" : ["destination_city"],
                    "additionalProperties": False
                }
            }
        return price_function

    
    def tool(self):
        tools = [{'type':'function', 'function': self.price_function()}]
        return tools


    def handle_tool_call(self, message):
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = self.get_ticket_prices()
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city
        
    
    def artist(self):
        image_response = openai.images.generate(
            model = 'dall-e-3',
            prompt=f"An image representing a vacation in {self.destination_city}, showing tourist spots and everything unique about {self.destination_city}, in a vibrant pop-art style",
                size="1024x1024",
                n=1,
                response_format="b64_json",
            )
        # print(image_response)
        image_base64 = image_response.data[0].b64_json
        image_data = base64.b64decode(image_base64)
        # print(image_data)
        return Image.open(BytesIO(image_data))

    
    def talker(self, message):
        response = openai.audio.speech.create(
          model="tts-1",
          voice="onyx",    # Also, try replacing onyx with alloy
          input=message
        )
        print(response)
        audio_stream = BytesIO(response.content)
        audio = AudioSegment.from_file(audio_stream, format="mp3")
        play(audio)


    def chat(self, history):
        messages = [{"role": "system", "content": system_message}] + history
        response = openai.chat.completions.create(model=openai_model, messages=messages, tools=tools)
        image = None
        
        if response.choices[0].finish_reason=="tool_calls":
            message = response.choices[0].message
            response, city = handle_tool_call(message)
            messages.append(message)
            messages.append(response)
            image = artist(self.destination_city)
            response = openai.chat.completions.create(model=openai_model, messages=messages)
            
        reply = response.choices[0].message.content
        history += [{"role":"assistant", "content":reply}]
    
        talker(reply)

        return history, image

    
    def ui(self):
        with gr.Blocks() as ui:
            with gr.Row():
                chatbot = gr.Chatbot(height=500, type="messages")
                image_output = gr.Image(height=500)
            with gr.Row():
                entry = gr.Textbox(label="Chat with our AI Assistant:")
            with gr.Row():
                clear = gr.Button("Clear")
        
            def do_entry(message, history):
                history += [{"role":"user", "content":message}]
                return "", history
        
            entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
                chat, inputs=chatbot, outputs=[chatbot, image_output]
            )
            clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)
        
        ui.launch(inbrowser=True)

In [72]:
assistant = AirlineAssistant('London')
assistant.ui()

* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


Input #0, wav, from '/var/folders/90/h9hy_l096_jcf3041fnkklp00000gn/T/tmp67d6hq_q.wav':
  Duration: 00:00:02.04, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.94 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

Input #0, wav, from '/var/folders/90/h9hy_l096_jcf3041fnkklp00000gn/T/tmps9_w1gdp.wav':
  Duration: 00:00:04.46, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   4.38 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 


Tool get_ticket_prices for London


Input #0, wav, from '/var/folders/90/h9hy_l096_jcf3041fnkklp00000gn/T/tmpexklailf.wav':
  Duration: 00:00:06.12, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   6.00 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 